# Replication Notebook

This notebook accompanies our paper and demonstrates, step by step, how the proposed methodology reproduces the main empirical findings. In particular, it serves three purposes:

1. **Replication of Figures:** 
   We begin by reproducing Figure 2 from the paper. This provides a visual check that our implementation matches the patterns reported in the published results.  

2. **Replication of Tables:**  
   Then, we replicate Table 1 and Table 2. These tables report the numerical results that summarize model performance, parameter estimates, and selection accuracy. Reproducing them ensures the pipeline is correct and demonstrates that our code can fully support the empirical analysis in the paper.  

---

This paper will not go over how the high dimensional and normal VECM algorithm work, for that please see the other (two) notebooks.
For example we wont show that all time series are I(1) every time.


First we will use the latent efficient bid price from section 4.2 of our paper, and then we will use section 4.3, the testing procedure for table 1 and table 2 is as described in 4.4

In [ ]:
#

## Testing the Common Trend Component for Accuracy

We will use Section 4.4 first.
To evaluate whether the extracted and scaled common trend component 
$\{ \widehat{y}_{t,j}^{(b)} \}_{t=1}^{T}$ (or the permanent component) captures meaningful structure in the underlying asset,  
we study whether the observed price series $\{ y_{t,j}^{(b)} \}_{t=1}^{T}$ tends to revert toward it after deviating by more than a relative threshold $\delta \in [0,1)$.  

Formally, we define a sequence of *crossing times* and then construct a metric that quantifies how often the observed price returns to its latent efficient price following a significant deviation.

---

### Defining the crossing sequence

Let $t = 1,\dots,T$ and define the sequence of indices $\{ i_k \}_{k=0}^{W}$ with $W > 0$, where each $i_k$ is the time of the next “crossing” event. We alternate between downward and upward deviations as follows:

- The first index $i_0$ is the earliest time where the observed price falls at least $\delta$ below its trend:
  $$
  i_0 = \inf \{ k \in [1,T] : y_{t,k}^{(b)} < \widehat{y}_{t,k}^{(b)} (1-\delta) \}.
  $$
- The next index $i_1$ is the first time after $i_0$ where the series crosses back above the trend:
  $$
  i_1 = \inf \{ k \in (i_0,T] : y_{t,k}^{(b)} \geq \widehat{y}_{t,k}^{(b)} \}.
  $$
- Then $i_2$ is defined analogously as the first time after $i_1$ where the series again drops at least $\delta$ below the trend, and so on.

In general,
$$
i_k = \inf \Bigg\{ n \in (i_{k-1},T] :
\begin{cases}
y_{t,n}^{(b)} < \widehat{y}_{t,n}^{(b)} (1-\delta), & k \ \text{even}, \\[6pt]
y_{t,n}^{(b)} \geq \widehat{y}_{t,n}^{(b)}, & k \ \text{odd}.
\end{cases}
\Bigg\}.
$$

The sequence terminates once $i_k \in \{T,\infty\}$. In that case, the last stopping time is defined as
$$
i_N \coloneqq
\begin{cases}
i_{k-1}, & k \ \text{even}, \\
i_k, & k \ \text{odd}.
\end{cases}
$$

By convention, $\inf\{\emptyset\} = \infty$, so the sequence $\{ i_k \}$ is always well-defined, even if some conditions are never met.

---

### Empirical reversion probability

We are interested in the conditional probability that the observed price moves upward once it crosses back above the trend after being at least $\delta$ below. Intuitively, this corresponds to the probability that an undervalued asset reverts toward equilibrium.

Formally,
$$
\mathbb{P}\!\left[
y_{i_{k+1},j}^{(b)} > y_{i_{k},j}^{(b)} ,\ 
y_{i_{k+1},j}^{(b)} \geq \widehat{y}_{i_{k+1},j}^{(b)}
\ \middle|\ 
y_{i_{k},j}^{(b)} < \widehat{y}_{i_{k},j}^{(b)} (1-\delta)
\right].
$$

By the definition of the stopping times, the additional conditions  
$y_{i_{k+1},j}^{(b)} \geq \widehat{y}_{i_{k+1},j}^{(b)}$ and  
$y_{i_{k},j}^{(b)} < \widehat{y}_{i_{k},j}^{(b)} (1-\delta)$  
are already satisfied, so this reduces to
$$
\mathbb{P}\!\left[ y_{i_{k+1},j}^{(b)} > y_{i_{k},j}^{(b)} \right].
$$

---

### Estimator

The empirical counterpart is

$$
\widehat{P}_{\uparrow}
= \frac{1}{N} \sum_{k=0}^{N-1}
\mathbf{1}\!\left\{\, y_{i_{k+1},j}^{(b)} > y_{i_{k},j}^{(b)} \,\right\},
$$

where  

- $\mathbf{1}\{\cdot\}$ denotes the indicator function,  
- $i_k$ and $i_{k+1}$ are consecutive crossing times (down $\to$ up),  
- $N$ is the total number of such events before termination.  

This statistic measures the fraction of downward deviations that are followed by an upward correction, quantifying the predictive content of the estimated common trend.

---

### Downward counterpart

Analogously, define $\widehat{P}_{\downarrow}$ for upward deviations beyond $(1+\delta)$ of the trend, which measures the frequency with which prices correct downward after significant positive departures.

Trivially we will use TP, TN, FP and FN to denote the probabilitys of reversion.

## Standard approach (3.3)

In [ ]:
import numpy as np
import pandas as pd

from group_lasso_prox_lag import GroupLassoProxLagSelection
from group_lasso_prox_rank import GroupLassoProxRankSelection
from helpers import Helpers
from multinomial_logit_model import MultinomialLogit
from vecm_model_high_dim import VECMModelHD
from vecm_model_standard import VECMModel

import warnings
from statsmodels.tools.sm_exceptions import HessianInversionWarning

warnings.simplefilter("ignore", HessianInversionWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
# The exchange names as hinted in footnote 1 of our paper
DF_NAMES = ['binance', 'bitmart', 'bybit', 'hashkey', 'htx', 'bitget', 'deribit', 'kuma', 'hyperliquid']
DF_NAMES_CLEAN = ['binance', 'bitmart', 'bybit', 'htx', 'bitget', 'deribit', 'kuma', 'hyperliquid']

# Keep track of the performance of every exchange respectively
score = {}
for exchange_name in DF_NAMES_CLEAN:
    score[exchange_name] = {}
    score[exchange_name]['BID_SIDE'] = {}
    score[exchange_name]['BID_SIDE']['TP'] = 0
    score[exchange_name]['BID_SIDE']['FP'] = 0
    score[exchange_name]['BID_SIDE']['TN'] = 0
    score[exchange_name]['BID_SIDE']['FN'] = 0

    score[exchange_name]['ASK_SIDE'] = {}
    score[exchange_name]['ASK_SIDE']['TP'] = 0
    score[exchange_name]['ASK_SIDE']['FP'] = 0
    score[exchange_name]['ASK_SIDE']['TN'] = 0
    score[exchange_name]['ASK_SIDE']['FN'] = 0

In [ ]:
# To reproduce Figure 2, one needs to load the wirst 50_000 rows on the csv file and use the following plotting method:

'''plt.plot(Y_b[0][:220], label='binance bids')
plt.plot(Y_b[1][:220], label='bitmart bids')
plt.plot(Y_b[2][:220], label='bybit bids')
plt.plot(Y_b[3][:220], label='htx bids')
plt.plot(Y_b[4][:220], label='bitget bids')
plt.plot(Y_b[5][:220], label='deribit bids')
plt.plot(Y_b[6][:220], label='kuma bids')
plt.plot(Y_b[7][:220], label='hyperliquid bids')

plt.xlabel("t")        # x-axis label
plt.ylabel("price")    # y-axis label
plt.legend()
plt.show()'''

### Using Section 4.2 (Granger Representation Theorem)

In [ ]:
# Window size
H = 50_000
# Train percentage
RHO = 0.7
# Max total length of the data we look at
max_j = 11964000 # pre determined number of rows of the file
# The stepsize
STEPSIZE = int((1-RHO)*H)
# Set the lag once
LAG = 20
# Set the rank once
RANK = 7 # K - 1
# Set threshold delta
DELTA = 0.0005
# Path
PATH = "/Downloads/data.csv"

for j in range(0, max_j, STEPSIZE):
    try:
        # This should be the path to the data_1.csv file on your local machine
        skip_rows = j
        n_rows = H

        VECM_model_b = VECMModel()
        VECM_model_a = VECMModel()
        helper_methods = Helpers()

        binance, bitmart, bybit, hashkey, htx, bitget, deribit, kuma, hyperliquid = helper_methods.read_csv_to_dataframe(PATH, skip_rows, n_rows, DF_NAMES)
        # We will skip hashkey exchange
        Y_b = VECM_model_b.construct_Y([binance.bids, bitmart.bids, bybit.bids, htx.bids, bitget.bids, deribit.bids, kuma.bids, hyperliquid.bids])
        Y_a = VECM_model_a.construct_Y([binance.asks, bitmart.asks, bybit.asks, htx.asks, bitget.asks, deribit.asks, kuma.asks, hyperliquid.asks])
        # Train test split
        Y_train_b, Y_test_b = VECM_model_b.train_test_split_Y(Y_b, RHO)
        Y_train_a, Y_test_a = VECM_model_a.train_test_split_Y(Y_a, RHO)
        # Set the lag
        VECM_model_b.lag = LAG
        VECM_model_a.lag = LAG
        # Build the VECM matrices
        VECM_model_b.build_vecm_matrices(Y_train_b)
        VECM_model_a.build_vecm_matrices(Y_train_a)
        # Build residual covaraince matrices
        VECM_model_b.build_residual_covariances()
        VECM_model_a.build_residual_covariances()
        # Build johansen matrix
        S_tilde_b = VECM_model_b.get_johansen_matrix()
        S_tilde_a = VECM_model_a.get_johansen_matrix()
        # Compute sorted eigenvalues
        eigvals_sorted_b, _ = VECM_model_b.sort_eigenvectors(S_tilde_b)
        eigvals_sorted_a, _ = VECM_model_a.sort_eigenvectors(S_tilde_a)
        # Set Rank
        VECM_model_b.rank = RANK
        VECM_model_a.rank = RANK
        # Compute the variables
        alpha_b, beta_b, Gammas_b, Sigma_u_b = VECM_model_b.vecm_variable_estimation()
        alpha_a, beta_a, Gammas_a, Sigma_u_a = VECM_model_a.vecm_variable_estimation()
        # Compute the orthogonal compoents of alpha and beta
        alpha_perp_b = VECM_model_b.compute_null_space_basis(alpha_b.T)
        beta_perp_b = VECM_model_b.compute_null_space_basis(beta_b.T)
        alpha_perp_a = VECM_model_a.compute_null_space_basis(alpha_a.T)
        beta_perp_a = VECM_model_a.compute_null_space_basis(beta_a.T)
        # First evaluate the latent efficient price from Section 4.2 of our paper
        Xi_b = VECM_model_b.compute_granger_representation_matrix_XI(alpha_perp_b, beta_perp_b, Gammas_b)
        Xi_a = VECM_model_a.compute_granger_representation_matrix_XI(alpha_perp_a, beta_perp_a, Gammas_a)

        # Loop over all variables to compute the score respectively
        for idx, exchange_name in zip(range(len(DF_NAMES_CLEAN)), DF_NAMES_CLEAN):
            Lambda_b = (Xi_b @ Y_train_b)[idx]
            Lambda_a = (Xi_b @ Y_train_b)[idx]

            y_b = Y_train_b[idx].T
            y_breve_b = Lambda_b

            y_a = Y_train_a[idx].T
            y_breve_a = Lambda_a
            # Compute scaling factor
            phi_b = VECM_model_b.estimate_phi(y_b, y_breve_b)
            phi_a = VECM_model_a.estimate_phi(y_a, y_breve_a)
            # Define efficient latent prices
            efficient_price_b = phi_b * (Xi_b @ Y_test_b)[idx]
            efficient_price_a = phi_a * (Xi_b @ Y_test_a)[idx]

            Y_test_b_idx = Y_test_b[idx]
            Y_test_a_idx = Y_test_a[idx]
            # Performance for bid series
            crossed_below_delta = np.where(Y_test_b_idx < efficient_price_b * (1 - DELTA))[0]
            crossed_above = np.where(Y_test_b_idx > efficient_price_b)[0]

            crossed_above_delta = np.where(Y_test_b_idx> efficient_price_b * (1 + DELTA))[0]
            crossed_below = np.where(Y_test_b_idx < efficient_price_b)[0]

            for i_k in crossed_below_delta:
                try:
                    i_k_1 = crossed_above[np.where(crossed_above > i_k)[0][0]]

                    difference = Y_test_b_idx[i_k_1] - Y_test_b_idx[i_k]
                    if difference > 0:
                        score[exchange_name]['BID_SIDE']['TP'] += 1
                    else:
                        score[exchange_name]['BID_SIDE']['FP'] += 1

                # Breaks when ther is no index left for wich np.where(... > i_k)[0][0] is true
                except:
                    break

            for i_k in crossed_above_delta:
                try:
                    i_k_1 = crossed_below[np.where(crossed_below > i_k)[0][0]]

                    difference = Y_test_b_idx[i_k] - Y_test_b_idx[i_k_1]
                    if difference > 0:
                        score[exchange_name]['BID_SIDE']['TN'] += 1
                    else:
                        score[exchange_name]['BID_SIDE']['FN'] += 1

                # Breaks when ther is no index left for wich np.where(... > i_k)[0][0] is true
                except:
                    break

            # Performance for ask series
            crossed_below_delta = np.where(Y_test_a_idx < efficient_price_a * (1 - DELTA))[0]
            crossed_above = np.where(Y_test_a_idx > efficient_price_a)[0]

            crossed_above_delta = np.where(Y_test_a_idx > efficient_price_a * (1 + DELTA))[0]
            crossed_below = np.where(Y_test_a_idx < efficient_price_a)[0]

            for i_k in crossed_below_delta:
                try:
                    i_k_1 = crossed_above[np.where(crossed_above > i_k)[0][0]]

                    difference = Y_test_a_idx[i_k_1] - Y_test_a_idx[i_k]
                    if difference > 0:
                        score[exchange_name]['ASK_SIDE']['TP'] += 1
                    elif difference < 0:
                        score[exchange_name]['ASK_SIDE']['FP'] += 1
                
                # Breaks when ther is no index left for wich np.where(... > i_k)[0][0] is true
                except:
                    break

            for i_k in crossed_above_delta:
                try:
                    i_k_1 = crossed_below[np.where(crossed_below > i_k)[0][0]]

                    difference = Y_test_a_idx[i_k] - Y_test_a_idx[i_k_1]
                    if difference > 0:
                        score[exchange_name]['ASK_SIDE']['TN'] += 1
                    elif difference < 0:
                        score[exchange_name]['ASK_SIDE']['FN'] += 1

                # Breaks when ther is no index left for wich np.where(... > i_k)[0][0] is true
                except:
                    break

        print(f"###{j / max_j}###")

    except Exception as exp:
        print(f"Something went wrong: {exp}")
        break

In [ ]:
# Print the percentages of the score
for exchange_name in DF_NAMES_CLEAN:
    accuracy_bid_side_up = score[exchange_name]['BID_SIDE']['TP'] / (score[exchange_name]['BID_SIDE']['TP'] + score[exchange_name]['BID_SIDE']['FP'])
    accuracy_ask_side_up = score[exchange_name]['ASK_SIDE']['TP'] / (score[exchange_name]['ASK_SIDE']['TP'] + score[exchange_name]['ASK_SIDE']['FP'])

    accuracy_bid_side_down = score[exchange_name]['BID_SIDE']['TN'] / (score[exchange_name]['BID_SIDE']['TN'] + score[exchange_name]['BID_SIDE']['FN'])
    accuracy_ask_side_down = score[exchange_name]['ASK_SIDE']['TN'] / (score[exchange_name]['ASK_SIDE']['TN'] + score[exchange_name]['ASK_SIDE']['FN'])

    print(f"{exchange_name} has accuracy of {accuracy_bid_side_up} for updwards correction on the bid side")
    print(f"{exchange_name} has accuracy of {accuracy_ask_side_up} for downwards correction on the ask side")

    print(f"{exchange_name} has accuracy of {accuracy_bid_side_down} for updwards correction on the bid side")
    print(f"{exchange_name} has accuracy of {accuracy_ask_side_down} for downwards correction on the ask side")
    print("\n")
        

### Using Section 4.3 (P-T Decomposition)

In [ ]:
# The exchange names as hinted in footnote 1 of our paper
DF_NAMES = ['binance', 'bitmart', 'bybit', 'hashkey', 'htx', 'bitget', 'deribit', 'kuma', 'hyperliquid']
DF_NAMES_CLEAN = ['binance', 'bitmart', 'bybit', 'htx', 'bitget', 'deribit', 'kuma', 'hyperliquid']

# Keep track of the performance of every exchange respectively
score = {}
for exchange_name in DF_NAMES_CLEAN:
    score[exchange_name] = {}
    score[exchange_name]['BID_SIDE'] = {}
    score[exchange_name]['BID_SIDE']['TP'] = 0
    score[exchange_name]['BID_SIDE']['FP'] = 0
    score[exchange_name]['BID_SIDE']['TN'] = 0
    score[exchange_name]['BID_SIDE']['FN'] = 0

    score[exchange_name]['ASK_SIDE'] = {}
    score[exchange_name]['ASK_SIDE']['TP'] = 0
    score[exchange_name]['ASK_SIDE']['FP'] = 0
    score[exchange_name]['ASK_SIDE']['TN'] = 0
    score[exchange_name]['ASK_SIDE']['FN'] = 0

In [ ]:
# Window size
H = 50_000
# Train percentage
RHO = 0.7
# Max total length of the data we look at
max_j = 11964000 # pre determined number of rows of the file
# The stepsize
STEPSIZE = int((1-RHO)*H)
# Set the lag once
LAG = 20
# Set the rank once
RANK = 7 # K - 1
# Set threshold delta
DELTA = 0.0005
# Path to the data
PATH = "/Downloads/data.csv"

for j in range(0, max_j, STEPSIZE):
    try:
        # This should be the path to the data_1.csv file on your local machine
        skip_rows = j
        n_rows = H

        VECM_model_b = VECMModel()
        VECM_model_a = VECMModel()
        helper_methods = Helpers()

        binance, bitmart, bybit, hashkey, htx, bitget, deribit, kuma, hyperliquid = helper_methods.read_csv_to_dataframe(PATH, skip_rows, n_rows, DF_NAMES)
        Y_b = VECM_model_b.construct_Y([binance.bids, bitmart.bids, bybit.bids, htx.bids, bitget.bids, deribit.bids, kuma.bids, hyperliquid.bids])
        Y_a = VECM_model_a.construct_Y([binance.asks, bitmart.asks, bybit.asks, htx.asks, bitget.asks, deribit.asks, kuma.asks, hyperliquid.asks])
        # Train test split
        Y_train_b, Y_test_b = VECM_model_b.train_test_split_Y(Y_b, RHO)
        Y_train_a, Y_test_a = VECM_model_a.train_test_split_Y(Y_a, RHO)
        # Set the lag
        VECM_model_b.lag = LAG
        VECM_model_a.lag = LAG
        # Build the VECM matrices
        VECM_model_b.build_vecm_matrices(Y_train_b)
        VECM_model_a.build_vecm_matrices(Y_train_a)
        # Build residual covaraince matrices
        VECM_model_b.build_residual_covariances()
        VECM_model_a.build_residual_covariances()
        # Build johansen matrix
        S_tilde_b = VECM_model_b.get_johansen_matrix()
        S_tilde_a = VECM_model_a.get_johansen_matrix()
        # Compute sorted eigenvalues
        eigvals_sorted_b, _ = VECM_model_b.sort_eigenvectors(S_tilde_b)
        eigvals_sorted_a, _ = VECM_model_a.sort_eigenvectors(S_tilde_a)
        # Set Rank
        VECM_model_b.rank = RANK
        VECM_model_a.rank = RANK
        # Compute the variables
        alpha_b, beta_b, Gammas_b, Sigma_u_b = VECM_model_b.vecm_variable_estimation()
        alpha_a, beta_a, Gammas_a, Sigma_u_a = VECM_model_a.vecm_variable_estimation()
        # Compute the orthogonal compoents of alpha and beta
        alpha_perp_b = VECM_model_b.compute_null_space_basis(alpha_b.T)
        beta_perp_b = VECM_model_b.compute_null_space_basis(beta_b.T)
        alpha_perp_a = VECM_model_a.compute_null_space_basis(alpha_a.T)
        beta_perp_a = VECM_model_a.compute_null_space_basis(beta_a.T)
        # First evaluate the latent efficient price from Section 4.2 of our paper
        persistent_component_b, transitory_component_b = VECM_model_b.compute_P_T_decomposition(alpha_b, beta_b, alpha_perp_b, beta_perp_b, Y_test_b)
        persistent_component_a, transitory_component_a = VECM_model_a.compute_P_T_decomposition(alpha_b, beta_b, alpha_perp_a, beta_perp_a, Y_test_a)

        # Loop over all variables to compute the score respectively
        for idx, exchange_name in zip(range(len(DF_NAMES_CLEAN)), DF_NAMES_CLEAN):
            # Define efficient latent prices
            efficient_price_b = persistent_component_b[idx]
            efficient_price_a = persistent_component_a[idx]

            Y_test_b_idx = Y_test_b[idx]
            Y_test_a_idx = Y_test_a[idx]
            # Performance for bid series
            crossed_below_delta = np.where(Y_test_b_idx < efficient_price_b * (1 - DELTA))[0]
            crossed_above = np.where(Y_test_b_idx > efficient_price_b)[0]

            crossed_above_delta = np.where(Y_test_b_idx > efficient_price_b * (1 + DELTA))[0]
            crossed_below = np.where(Y_test_b_idx < efficient_price_b)[0]

            for i_k in crossed_below_delta:
                try:
                    i_k_1 = crossed_above[np.where(crossed_above > i_k)[0][0]]

                    difference = Y_test_b_idx[i_k_1] - Y_test_b_idx[i_k]
                    if difference > 0:
                        score[exchange_name]['BID_SIDE']['TP'] += 1
                    else:
                        score[exchange_name]['BID_SIDE']['FP'] += 1

                # Breaks when ther is no index left for wich np.where(... > i_k)[0][0] is true
                except:
                    break

            for i_k in crossed_above_delta:
                try:
                    i_k_1 = crossed_below[np.where(crossed_below > i_k)[0][0]]

                    difference = Y_test_b_idx[i_k] - Y_test_b_idx[i_k_1]
                    if difference > 0:
                        score[exchange_name]['BID_SIDE']['TN'] += 1
                    else:
                        score[exchange_name]['BID_SIDE']['FN'] += 1

                # Breaks when ther is no index left for wich np.where(... > i_k)[0][0] is true
                except:
                    break

            # Performance for ask series
            crossed_below_delta = np.where(Y_test_a_idx < efficient_price_a * (1 - DELTA))[0]
            crossed_above = np.where(Y_test_a_idx > efficient_price_a)[0]

            crossed_above_delta = np.where(Y_test_a_idx > efficient_price_a * (1 + DELTA))[0]
            crossed_below = np.where(Y_test_a_idx < efficient_price_a)[0]

            for i_k in crossed_below_delta:
                try:
                    i_k_1 = crossed_above[np.where(crossed_above > i_k)[0][0]]

                    difference = Y_test_a_idx[i_k_1] - Y_test_a_idx[i_k]
                    if difference > 0:
                        score[exchange_name]['ASK_SIDE']['TP'] += 1
                    else:
                        score[exchange_name]['ASK_SIDE']['FP'] += 1
                
                # Breaks when ther is no index left for wich np.where(... > i_k)[0][0] is true
                except:
                    break

            for i_k in crossed_above_delta:
                try:
                    i_k_1 = crossed_below[np.where(crossed_below > i_k)[0][0]]

                    difference = Y_test_a_idx[i_k] - Y_test_a_idx[i_k_1]
                    if difference > 0:
                        score[exchange_name]['ASK_SIDE']['TN'] += 1
                    else:
                        score[exchange_name]['ASK_SIDE']['FN'] += 1

                # Breaks when ther is no index left for wich np.where(... > i_k)[0][0] is true
                except:
                    break

        print(f"###{j / max_j}###")
    
    except Exception as exp:
        print(f"Something went wrong: {exp}")
        break

In [ ]:
# Print the percentages of the score
for exchange_name in DF_NAMES_CLEAN:
    accuracy_bid_side_up = score[exchange_name]['BID_SIDE']['TP'] / (score[exchange_name]['BID_SIDE']['TP'] + score[exchange_name]['BID_SIDE']['FP'])
    accuracy_ask_side_up = score[exchange_name]['ASK_SIDE']['TP'] / (score[exchange_name]['ASK_SIDE']['TP'] + score[exchange_name]['ASK_SIDE']['FP'])

    accuracy_bid_side_down = score[exchange_name]['BID_SIDE']['TN'] / (score[exchange_name]['BID_SIDE']['TN'] + score[exchange_name]['BID_SIDE']['FN'])
    accuracy_ask_side_down = score[exchange_name]['ASK_SIDE']['TN'] / (score[exchange_name]['ASK_SIDE']['TN'] + score[exchange_name]['ASK_SIDE']['FN'])

    print(f"{exchange_name} has accuracy of {accuracy_bid_side_up} for updwards correction on the bid side")
    print(f"{exchange_name} has accuracy of {accuracy_ask_side_up} for downwards correction on the ask side")

    print(f"{exchange_name} has accuracy of {accuracy_bid_side_down} for updwards correction on the bid side")
    print(f"{exchange_name} has accuracy of {accuracy_ask_side_down} for downwards correction on the ask side")
    print("\n")
        

## High Dimensional approach (3.4)

In [ ]:
# The exchange names as hinted in footnote 1 of our paper
DF_NAMES = ['binance', 'bitmart', 'bybit', 'hashkey', 'htx', 'bitget', 'deribit', 'kuma', 'hyperliquid']
DF_NAMES_CLEAN = ['binance', 'bitmart', 'bybit', 'htx', 'bitget', 'deribit', 'kuma', 'hyperliquid']

# Keep track of the performance of every exchange respectively
score = {}
for exchange_name in DF_NAMES_CLEAN:
    score[exchange_name] = {}
    score[exchange_name]['BID_SIDE'] = {}
    score[exchange_name]['BID_SIDE']['TP'] = 0
    score[exchange_name]['BID_SIDE']['FP'] = 0
    score[exchange_name]['BID_SIDE']['TN'] = 0
    score[exchange_name]['BID_SIDE']['FN'] = 0

    score[exchange_name]['ASK_SIDE'] = {}
    score[exchange_name]['ASK_SIDE']['TP'] = 0
    score[exchange_name]['ASK_SIDE']['FP'] = 0
    score[exchange_name]['ASK_SIDE']['TN'] = 0
    score[exchange_name]['ASK_SIDE']['FN'] = 0

### Using Section 4.2 (Granger Representation Theorem)

In [ ]:
# Window size
H = 50_000
# Train percentage
RHO = 0.7
# Max total length of the data we look at
max_j = 11964000 # pre determined number of rows of the file
# The stepsize
STEPSIZE = int((1-RHO)*H)
# Set the lag once
LAG = 20
# Set the rank once
RANK = 7 # K - 1
# Set threshold delta
DELTA = 0.0005
# Path
PATH = "/Downloads/data.csv"
# Adaptive group lasso solvers
group_lasso_prox_rank_b = GroupLassoProxRankSelection()
group_lasso_prox_rank_a = GroupLassoProxRankSelection()
group_lasso_prox_b = GroupLassoProxLagSelection()
group_lasso_prox_a = GroupLassoProxLagSelection()
# Define tau rank, tau lag and tau ridge and gamma
tau_rank_b = 1e10
tau_rank_a = 1e10
tau_lag_b = 0.01
tau_lag_a = 0.01
gamma = 2

for j in range(int(0.40496489468405217 * 11964000), max_j, STEPSIZE):
    try:
        # This should be the path to the data_1.csv file on your local machine
        skip_rows = j
        n_rows = H

        VECM_model_b = VECMModelHD()
        VECM_model_a = VECMModelHD()
        helper_methods = Helpers()

        binance, bitmart, bybit, hashkey, htx, bitget, deribit, kuma, hyperliquid = helper_methods.read_csv_to_dataframe(PATH, skip_rows, n_rows, DF_NAMES)
        # We will skip hashkey exchange
        Y_b = VECM_model_b.construct_Y([binance.bids, bitmart.bids, bybit.bids, htx.bids, bitget.bids, deribit.bids, kuma.bids, hyperliquid.bids])
        Y_a = VECM_model_a.construct_Y([binance.asks, bitmart.asks, bybit.asks, htx.asks, bitget.asks, deribit.asks, kuma.asks, hyperliquid.asks])
        # Train test split
        Y_train_b, Y_test_b = VECM_model_b.train_test_split_Y(Y_b, RHO)
        Y_train_a, Y_test_a = VECM_model_a.train_test_split_Y(Y_a, RHO)
        # Set the lag
        VECM_model_b.lag = LAG
        VECM_model_a.lag = LAG
        # Build the VECM matrices
        VECM_model_b.build_vecm_matrices(Y_train_b)
        VECM_model_a.build_vecm_matrices(Y_train_a)
        # Build frisch waugh rank matrices
        dY_tilde_b, Y1_tilde_b, Pi_tilde_b = VECM_model_b.frisch_waugh_rank_matrices()
        dY_tilde_a, Y1_tilde_a, Pi_tilde_a = VECM_model_a.frisch_waugh_rank_matrices()

        # Compute qr decomp
        Q_hat_b, R_tilde_b, Perm_b = VECM_model_b.qr_decomp(Pi_tilde_b.T)
        Q_hat_a, R_tilde_a, Perm_a = VECM_model_a.qr_decomp(Pi_tilde_a.T)
        # Compute permuation matrices
        Pmat_b = np.eye(Perm_b.shape[0])[:, Perm_b]
        Pmat_a = np.eye(Perm_a.shape[0])[:, Perm_a]
        # Compute mu_tildes
        mu_tilde_b = VECM_model_b.compute_mu_tilde(R_tilde_b)
        mu_tilde_a = VECM_model_a.compute_mu_tilde(R_tilde_a)
        # Rank pre estimate
        r_hat_qr_decomp_b = VECM_model_b.rank_pre_estimate(mu_tilde_b)
        r_hat_qr_decomp_a = VECM_model_a.rank_pre_estimate(mu_tilde_a)
        # Betas
        beta_b = VECM_model_b.get_beta(Q_hat_b, r_hat_qr_decomp_b)
        beta_a = VECM_model_a.get_beta(Q_hat_a, r_hat_qr_decomp_a)
        # Regressor matrices
        dY_tilde_b = Pmat_b.T @ dY_tilde_b
        dY_tilde_a = Pmat_a.T @ dY_tilde_a

        Y1_tilde_b = Pmat_b.T @ Y1_tilde_b
        Y1_tilde_a = Pmat_a.T @ Y1_tilde_a

        T = VECM_model_b.T_eff
        K = VECM_model_b.K

        weights = group_lasso_prox_rank_b.compute_weights(mu_tilde_b, gamma)
        weights = group_lasso_prox_rank_b.scale_weights(weights)
        Y, Z = group_lasso_prox_rank_b.construct_Y_Z_matrices(Q_hat_b, Y1_tilde_b, dY_tilde_b)

        R_hat_b, loss_history = group_lasso_prox_rank_b.fit(R_tilde_b, Y, Z, tau_rank_b, weights, tol = 1e-20)

        Sigma_w_b = VECM_model_b.residual_covariance(dY_tilde_b, R_hat_b.T @ Q_hat_b.T @ Y1_tilde_b)

        group_lasso_prox_rank_a = GroupLassoProxRankSelection()
        weights = group_lasso_prox_rank_a.compute_weights(mu_tilde_a, gamma)
        weights = group_lasso_prox_rank_a.scale_weights(weights)
        Y, Z = group_lasso_prox_rank_a.construct_Y_Z_matrices(Q_hat_a, Y1_tilde_a, dY_tilde_a)
        R_hat_a, loss_history = group_lasso_prox_rank_a.fit(R_tilde_a, Y, Z, tau_rank_a, weights, tol = 1e-20)

        Sigma_w_a = VECM_model_a.residual_covariance(dY_tilde_a, R_hat_a.T @ Q_hat_a.T @ Y1_tilde_a)

        # Set rank
        VECM_model_b.rank = RANK
        VECM_model_a.rank = RANK
        # Get alphas
        alpha_b = VECM_model_b.get_alpha(Pmat_b, R_hat_b, r_hat_qr_decomp_b)
        alpha_a = VECM_model_a.get_alpha(Pmat_a, R_hat_a, r_hat_qr_decomp_a)
        # Frisch waugh lag matrices
        dY_check_b, dX_check_b = VECM_model_b.frisch_waugh_lag_matrices()
        dY_check_a, dX_check_a = VECM_model_a.frisch_waugh_lag_matrices()
        # Gammas pre estimates
        Gamma_check_b = VECM_model_b.least_squares_estimate_Gamma(dX_check_b, dY_check_b)
        Gamma_check_a = VECM_model_a.least_squares_estimate_Gamma(dX_check_a, dY_check_a)
        c_b = 1
        c_a = 1
        tau_ridge_b = c_b * np.log(T)
        tau_ridge_a = c_a * np.log(T)
        Gamma_tilde_b = VECM_model_b.ridge_estimator_Gamma(dX_check_b, dY_check_b, tau_ridge_b, LAG)
        Gamma_tilde_a = VECM_model_a.ridge_estimator_Gamma(dX_check_a, dY_check_a, tau_ridge_a, LAG)

        # lag estimation
        K = VECM_model_b.K
        weights = group_lasso_prox_b.compute_weights(Gamma_tilde_b, LAG, K, gamma)
        weights = group_lasso_prox_b.scale_weights(weights)
        Y, Z = group_lasso_prox_b.helper_loss(LAG, dY_check_b)
        Gamma_hat_b, loss_history = group_lasso_prox_b.fit(Gamma_tilde_b, Y, Z, LAG, K, tau_lag_b, weights)
        Sigma_w_b = VECM_model_b.residual_covariance(dY_check_b, Gamma_hat_b @ dX_check_b)


        weights = group_lasso_prox_a.compute_weights(Gamma_tilde_a, LAG, K, gamma)
        weights = group_lasso_prox_a.scale_weights(weights)
        Y, Z = group_lasso_prox_a.helper_loss(LAG, dY_check_a)
        Gamma_hat_a, loss_history = group_lasso_prox_a.fit(Gamma_tilde_a, Y, Z, LAG, K, tau_lag_a, weights)
        Sigma_w_a = VECM_model_a.residual_covariance(dY_check_a, Gamma_hat_a @ dX_check_a)

        # set gamma and lag
        Gammas_b = Gamma_hat_b[:, :VECM_model_b.lag*K]
        Gammas_a = Gamma_hat_a[:, :VECM_model_a.lag*K]

        VECM_model_b.lag = LAG
        VECM_model_a.lag = LAG

        # Compute the orthogonal compoents of alpha and beta
        alpha_perp_b = VECM_model_b.compute_null_space_basis(alpha_b.T)
        beta_perp_b = VECM_model_b.compute_null_space_basis(beta_b.T)
        alpha_perp_a = VECM_model_a.compute_null_space_basis(alpha_a.T)
        beta_perp_a = VECM_model_a.compute_null_space_basis(beta_a.T)
        # First evaluate the latent efficient price from Section 4.2 of our paper
        Xi_b = VECM_model_b.compute_granger_representation_matrix_XI(alpha_perp_b, beta_perp_b, Gammas_b)
        Xi_a = VECM_model_a.compute_granger_representation_matrix_XI(alpha_perp_a, beta_perp_a, Gammas_a)

        # Loop over all variables to compute the score respectively
        for idx, exchange_name in zip(range(len(DF_NAMES_CLEAN)), DF_NAMES_CLEAN):
            Lambda_b = (Xi_b @ Y_train_b)[idx]
            Lambda_a = (Xi_b @ Y_train_b)[idx]

            y_b = Y_train_b[idx].T
            y_breve_b = Lambda_b

            y_a = Y_train_a[idx].T
            y_breve_a = Lambda_a
            # Compute scaling factor
            phi_b = VECM_model_b.estimate_phi(y_b, y_breve_b)
            phi_a = VECM_model_a.estimate_phi(y_a, y_breve_a)
            # Define efficient latent prices
            efficient_price_b = phi_b * (Xi_b @ Y_test_b)[idx]
            efficient_price_a = phi_a * (Xi_b @ Y_test_a)[idx]

            Y_test_b_idx = Y_test_b[idx]
            Y_test_a_idx = Y_test_a[idx]
            # Performance for bid series
            crossed_below_delta = np.where(Y_test_b_idx < efficient_price_b * (1 - DELTA))[0]
            crossed_above = np.where(Y_test_b_idx > efficient_price_b)[0]

            crossed_above_delta = np.where(Y_test_b_idx> efficient_price_b * (1 + DELTA))[0]
            crossed_below = np.where(Y_test_b_idx < efficient_price_b)[0]

            for i_k in crossed_below_delta:
                try:
                    i_k_1 = crossed_above[np.where(crossed_above > i_k)[0][0]]

                    difference = Y_test_b_idx[i_k_1] - Y_test_b_idx[i_k]
                    if difference > 0:
                        score[exchange_name]['BID_SIDE']['TP'] += 1
                    else:
                        score[exchange_name]['BID_SIDE']['FP'] += 1

                # Breaks when ther is no index left for wich np.where(... > i_k)[0][0] is true
                except:
                    break

            for i_k in crossed_above_delta:
                try:
                    i_k_1 = crossed_below[np.where(crossed_below > i_k)[0][0]]

                    difference = Y_test_b_idx[i_k] - Y_test_b_idx[i_k_1]
                    if difference > 0:
                        score[exchange_name]['BID_SIDE']['TN'] += 1
                    else:
                        score[exchange_name]['BID_SIDE']['FN'] += 1

                # Breaks when ther is no index left for wich np.where(... > i_k)[0][0] is true
                except:
                    break

            # Performance for ask series
            crossed_below_delta = np.where(Y_test_a_idx < efficient_price_a * (1 - DELTA))[0]
            crossed_above = np.where(Y_test_a_idx > efficient_price_a)[0]

            crossed_above_delta = np.where(Y_test_a_idx > efficient_price_a * (1 + DELTA))[0]
            crossed_below = np.where(Y_test_a_idx < efficient_price_a)[0]

            for i_k in crossed_below_delta:
                try:
                    i_k_1 = crossed_above[np.where(crossed_above > i_k)[0][0]]

                    difference = Y_test_a_idx[i_k_1] - Y_test_a_idx[i_k]
                    if difference > 0:
                        score[exchange_name]['ASK_SIDE']['TP'] += 1
                    elif difference < 0:
                        score[exchange_name]['ASK_SIDE']['FP'] += 1
                
                # Breaks when ther is no index left for wich np.where(... > i_k)[0][0] is true
                except:
                    break

            for i_k in crossed_above_delta:
                try:
                    i_k_1 = crossed_below[np.where(crossed_below > i_k)[0][0]]

                    difference = Y_test_a_idx[i_k] - Y_test_a_idx[i_k_1]
                    if difference > 0:
                        score[exchange_name]['ASK_SIDE']['TN'] += 1
                    elif difference < 0:
                        score[exchange_name]['ASK_SIDE']['FN'] += 1

                # Breaks when ther is no index left for wich np.where(... > i_k)[0][0] is true
                except:
                    break

        print(f"###{j / max_j}###")

    except Exception as exp:
        print(f"Something went wrong: {exp}")
        break

In [ ]:
# Print the percentages of the score
for exchange_name in DF_NAMES_CLEAN:
    accuracy_bid_side_up = score[exchange_name]['BID_SIDE']['TP'] / (score[exchange_name]['BID_SIDE']['TP'] + score[exchange_name]['BID_SIDE']['FP'])
    accuracy_ask_side_up = score[exchange_name]['ASK_SIDE']['TP'] / (score[exchange_name]['ASK_SIDE']['TP'] + score[exchange_name]['ASK_SIDE']['FP'])

    accuracy_bid_side_down = score[exchange_name]['BID_SIDE']['TN'] / (score[exchange_name]['BID_SIDE']['TN'] + score[exchange_name]['BID_SIDE']['FN'])
    accuracy_ask_side_down = score[exchange_name]['ASK_SIDE']['TN'] / (score[exchange_name]['ASK_SIDE']['TN'] + score[exchange_name]['ASK_SIDE']['FN'])

    print(f"{exchange_name} has accuracy of {accuracy_bid_side_up} for updwards correction on the bid side")
    print(f"{exchange_name} has accuracy of {accuracy_ask_side_up} for downwards correction on the ask side")

    print(f"{exchange_name} has accuracy of {accuracy_bid_side_down} for updwards correction on the bid side")
    print(f"{exchange_name} has accuracy of {accuracy_ask_side_down} for downwards correction on the ask side")
    print("\n")
        

### Using Section 4.3 (P-T Decomposition)

In [ ]:
# The exchange names as hinted in footnote 1 of our paper
DF_NAMES = ['binance', 'bitmart', 'bybit', 'hashkey', 'htx', 'bitget', 'deribit', 'kuma', 'hyperliquid']
DF_NAMES_CLEAN = ['binance', 'bitmart', 'bybit', 'htx', 'bitget', 'deribit', 'kuma', 'hyperliquid']

# Keep track of the performance of every exchange respectively
score = {}
for exchange_name in DF_NAMES_CLEAN:
    score[exchange_name] = {}
    score[exchange_name]['BID_SIDE'] = {}
    score[exchange_name]['BID_SIDE']['TP'] = 0
    score[exchange_name]['BID_SIDE']['FP'] = 0
    score[exchange_name]['BID_SIDE']['TN'] = 0
    score[exchange_name]['BID_SIDE']['FN'] = 0

    score[exchange_name]['ASK_SIDE'] = {}
    score[exchange_name]['ASK_SIDE']['TP'] = 0
    score[exchange_name]['ASK_SIDE']['FP'] = 0
    score[exchange_name]['ASK_SIDE']['TN'] = 0
    score[exchange_name]['ASK_SIDE']['FN'] = 0

In [ ]:
# Window size
H = 50_000
# Train percentage
RHO = 0.7
# Max total length of the data we look at
max_j = 11964000 # pre determined number of rows of the file
# The stepsize
STEPSIZE = int((1-RHO)*H)
# Set the lag once
LAG = 20
# Set the rank once
RANK = 7 # K - 1
# Set threshold delta
DELTA = 0.0005
# Path
PATH = "/Downloads/data.csv"
# Adaptive group lasso solvers
group_lasso_prox_rank_b = GroupLassoProxRankSelection()
group_lasso_prox_rank_a = GroupLassoProxRankSelection()
group_lasso_prox_b = GroupLassoProxLagSelection()
group_lasso_prox_a = GroupLassoProxLagSelection()
# Define tau rank, tau lag and tau ridge and gamma
tau_rank_b = 1e10
tau_rank_a = 1e10
tau_lag_b = 0.01
tau_lag_a = 0.01
gamma = 2

for j in range(0, max_j, STEPSIZE):
    try:
        # This should be the path to the data_1.csv file on your local machine
        skip_rows = j
        n_rows = H

        VECM_model_b = VECMModelHD()
        VECM_model_a = VECMModelHD()
        helper_methods = Helpers()

        binance, bitmart, bybit, hashkey, htx, bitget, deribit, kuma, hyperliquid = helper_methods.read_csv_to_dataframe(PATH, skip_rows, n_rows, DF_NAMES)
        # We will skip hashkey exchange
        Y_b = VECM_model_b.construct_Y([binance.bids, bitmart.bids, bybit.bids, htx.bids, bitget.bids, deribit.bids, kuma.bids, hyperliquid.bids])
        Y_a = VECM_model_a.construct_Y([binance.asks, bitmart.asks, bybit.asks, htx.asks, bitget.asks, deribit.asks, kuma.asks, hyperliquid.asks])
        # Train test split
        Y_train_b, Y_test_b = VECM_model_b.train_test_split_Y(Y_b, RHO)
        Y_train_a, Y_test_a = VECM_model_a.train_test_split_Y(Y_a, RHO)
        # Set the lag
        VECM_model_b.lag = LAG
        VECM_model_a.lag = LAG
        # Build the VECM matrices
        VECM_model_b.build_vecm_matrices(Y_train_b)
        VECM_model_a.build_vecm_matrices(Y_train_a)
        # Build frisch waugh rank matrices
        dY_tilde_b, Y1_tilde_b, Pi_tilde_b = VECM_model_b.frisch_waugh_rank_matrices()
        dY_tilde_a, Y1_tilde_a, Pi_tilde_a = VECM_model_a.frisch_waugh_rank_matrices()

        # Compute qr decomp
        Q_hat_b, R_tilde_b, Perm_b = VECM_model_b.qr_decomp(Pi_tilde_b.T)
        Q_hat_a, R_tilde_a, Perm_a = VECM_model_a.qr_decomp(Pi_tilde_a.T)
        # Compute permuation matrices
        Pmat_b = np.eye(Perm_b.shape[0])[:, Perm_b]
        Pmat_a = np.eye(Perm_a.shape[0])[:, Perm_a]
        # Compute mu_tildes
        mu_tilde_b = VECM_model_b.compute_mu_tilde(R_tilde_b)
        mu_tilde_a = VECM_model_a.compute_mu_tilde(R_tilde_a)
        # Rank pre estimate
        r_hat_qr_decomp_b = VECM_model_b.rank_pre_estimate(mu_tilde_b)
        r_hat_qr_decomp_a = VECM_model_a.rank_pre_estimate(mu_tilde_a)
        # Betas
        beta_b = VECM_model_b.get_beta(Q_hat_b, r_hat_qr_decomp_b)
        beta_a = VECM_model_a.get_beta(Q_hat_a, r_hat_qr_decomp_a)
        # Regressor matrices
        dY_tilde_b = Pmat_b.T @ dY_tilde_b
        dY_tilde_a = Pmat_a.T @ dY_tilde_a

        Y1_tilde_b = Pmat_b.T @ Y1_tilde_b
        Y1_tilde_a = Pmat_a.T @ Y1_tilde_a

        T = VECM_model_b.T_eff
        K = VECM_model_b.K

        weights = group_lasso_prox_rank_b.compute_weights(mu_tilde_b, gamma)
        weights = group_lasso_prox_rank_b.scale_weights(weights)
        Y, Z = group_lasso_prox_rank_b.construct_Y_Z_matrices(Q_hat_b, Y1_tilde_b, dY_tilde_b)

        R_hat_b, loss_history = group_lasso_prox_rank_b.fit(R_tilde_b, Y, Z, tau_rank_b, weights, tol = 1e-20)

        Sigma_w_b = VECM_model_b.residual_covariance(dY_tilde_b, R_hat_b.T @ Q_hat_b.T @ Y1_tilde_b)

        group_lasso_prox_rank_a = GroupLassoProxRankSelection()
        weights = group_lasso_prox_rank_a.compute_weights(mu_tilde_a, gamma)
        weights = group_lasso_prox_rank_a.scale_weights(weights)
        Y, Z = group_lasso_prox_rank_a.construct_Y_Z_matrices(Q_hat_a, Y1_tilde_a, dY_tilde_a)
        R_hat_a, loss_history = group_lasso_prox_rank_a.fit(R_tilde_a, Y, Z, tau_rank_a, weights, tol = 1e-20)

        Sigma_w_a = VECM_model_a.residual_covariance(dY_tilde_a, R_hat_a.T @ Q_hat_a.T @ Y1_tilde_a)

        # Set rank
        VECM_model_b.rank = RANK
        VECM_model_a.rank = RANK
        # Get alphas
        alpha_b = VECM_model_b.get_alpha(Pmat_b, R_hat_b, r_hat_qr_decomp_b)
        alpha_a = VECM_model_a.get_alpha(Pmat_a, R_hat_a, r_hat_qr_decomp_a)
        # Frisch waugh lag matrices
        dY_check_b, dX_check_b = VECM_model_b.frisch_waugh_lag_matrices()
        dY_check_a, dX_check_a = VECM_model_a.frisch_waugh_lag_matrices()
        # Gammas pre estimates
        Gamma_check_b = VECM_model_b.least_squares_estimate_Gamma(dX_check_b, dY_check_b)
        Gamma_check_a = VECM_model_a.least_squares_estimate_Gamma(dX_check_a, dY_check_a)
        c_b = 1
        c_a = 1
        tau_ridge_b = c_b * np.log(T)
        tau_ridge_a = c_a * np.log(T)
        Gamma_tilde_b = VECM_model_b.ridge_estimator_Gamma(dX_check_b, dY_check_b, tau_ridge_b, LAG)
        Gamma_tilde_a = VECM_model_a.ridge_estimator_Gamma(dX_check_a, dY_check_a, tau_ridge_a, LAG)

        # lag estimation
        K = VECM_model_b.K
        weights = group_lasso_prox_b.compute_weights(Gamma_tilde_b, LAG, K, gamma)
        weights = group_lasso_prox_b.scale_weights(weights)
        Y, Z = group_lasso_prox_b.helper_loss(LAG, dY_check_b)
        Gamma_hat_b, loss_history = group_lasso_prox_b.fit(Gamma_tilde_b, Y, Z, LAG, K, tau_lag_b, weights)
        Sigma_w_b = VECM_model_b.residual_covariance(dY_check_b, Gamma_hat_b @ dX_check_b)


        weights = group_lasso_prox_a.compute_weights(Gamma_tilde_a, LAG, K, gamma)
        weights = group_lasso_prox_a.scale_weights(weights)
        Y, Z = group_lasso_prox_a.helper_loss(LAG, dY_check_a)
        Gamma_hat_a, loss_history = group_lasso_prox_a.fit(Gamma_tilde_a, Y, Z, LAG, K, tau_lag_a, weights)
        Sigma_w_a = VECM_model_a.residual_covariance(dY_check_a, Gamma_hat_a @ dX_check_a)

        # set gamma and lag
        Gammas_b = Gamma_hat_b[:, :VECM_model_b.lag*K]
        Gammas_a = Gamma_hat_a[:, :VECM_model_a.lag*K]

        VECM_model_b.lag = LAG
        VECM_model_a.lag = LAG

        # Compute the orthogonal compoents of alpha and beta
        alpha_perp_b = VECM_model_b.compute_null_space_basis(alpha_b.T)
        beta_perp_b = VECM_model_b.compute_null_space_basis(beta_b.T)
        alpha_perp_a = VECM_model_a.compute_null_space_basis(alpha_a.T)
        beta_perp_a = VECM_model_a.compute_null_space_basis(beta_a.T)
        # First evaluate the latent efficient price from Section 4.2 of our paper
        persistent_component_b, transitory_component_b = VECM_model_b.compute_P_T_decomposition(alpha_b, beta_b, alpha_perp_b, beta_perp_b, Y_test_b)
        persistent_component_a, transitory_component_a = VECM_model_a.compute_P_T_decomposition(alpha_b, beta_b, alpha_perp_a, beta_perp_a, Y_test_a)


        # Loop over all variables to compute the score respectively
        for idx, exchange_name in zip(range(len(DF_NAMES_CLEAN)), DF_NAMES_CLEAN):
            # Define efficient latent prices
            efficient_price_b = persistent_component_b[idx]
            efficient_price_a = persistent_component_a[idx]

            Y_test_b_idx = Y_test_b[idx]
            Y_test_a_idx = Y_test_a[idx]
            # Performance for bid series
            crossed_below_delta = np.where(Y_test_b_idx < efficient_price_b * (1 - DELTA))[0]
            crossed_above = np.where(Y_test_b_idx > efficient_price_b)[0]

            crossed_above_delta = np.where(Y_test_b_idx > efficient_price_b * (1 + DELTA))[0]
            crossed_below = np.where(Y_test_b_idx < efficient_price_b)[0]

            for i_k in crossed_below_delta:
                try:
                    i_k_1 = crossed_above[np.where(crossed_above > i_k)[0][0]]

                    difference = Y_test_b_idx[i_k_1] - Y_test_b_idx[i_k]
                    if difference > 0:
                        score[exchange_name]['BID_SIDE']['TP'] += 1
                    else:
                        score[exchange_name]['BID_SIDE']['FP'] += 1

                # Breaks when ther is no index left for wich np.where(... > i_k)[0][0] is true
                except:
                    break

            for i_k in crossed_above_delta:
                try:
                    i_k_1 = crossed_below[np.where(crossed_below > i_k)[0][0]]

                    difference = Y_test_b_idx[i_k] - Y_test_b_idx[i_k_1]
                    if difference > 0:
                        score[exchange_name]['BID_SIDE']['TN'] += 1
                    else:
                        score[exchange_name]['BID_SIDE']['FN'] += 1

                # Breaks when ther is no index left for wich np.where(... > i_k)[0][0] is true
                except:
                    break

            # Performance for ask series
            crossed_below_delta = np.where(Y_test_a_idx < efficient_price_a * (1 - DELTA))[0]
            crossed_above = np.where(Y_test_a_idx > efficient_price_a)[0]

            crossed_above_delta = np.where(Y_test_a_idx > efficient_price_a * (1 + DELTA))[0]
            crossed_below = np.where(Y_test_a_idx < efficient_price_a)[0]

            for i_k in crossed_below_delta:
                try:
                    i_k_1 = crossed_above[np.where(crossed_above > i_k)[0][0]]

                    difference = Y_test_a_idx[i_k_1] - Y_test_a_idx[i_k]
                    if difference > 0:
                        score[exchange_name]['ASK_SIDE']['TP'] += 1
                    elif difference < 0:
                        score[exchange_name]['ASK_SIDE']['FP'] += 1
                
                # Breaks when ther is no index left for wich np.where(... > i_k)[0][0] is true
                except:
                    break

            for i_k in crossed_above_delta:
                try:
                    i_k_1 = crossed_below[np.where(crossed_below > i_k)[0][0]]

                    difference = Y_test_a_idx[i_k] - Y_test_a_idx[i_k_1]
                    if difference > 0:
                        score[exchange_name]['ASK_SIDE']['TN'] += 1
                    elif difference < 0:
                        score[exchange_name]['ASK_SIDE']['FN'] += 1

                # Breaks when ther is no index left for wich np.where(... > i_k)[0][0] is true
                except:
                    break

        print(f"###{j / max_j}###")

    except Exception as exp:
        print(f"Something went wrong: {exp}")
        break

In [ ]:
# Print the percentages of the score
for exchange_name in DF_NAMES_CLEAN:
    accuracy_bid_side_up = score[exchange_name]['BID_SIDE']['TP'] / (score[exchange_name]['BID_SIDE']['TP'] + score[exchange_name]['BID_SIDE']['FP'])
    accuracy_ask_side_up = score[exchange_name]['ASK_SIDE']['TP'] / (score[exchange_name]['ASK_SIDE']['TP'] + score[exchange_name]['ASK_SIDE']['FP'])

    accuracy_bid_side_down = score[exchange_name]['BID_SIDE']['TN'] / (score[exchange_name]['BID_SIDE']['TN'] + score[exchange_name]['BID_SIDE']['FN'])
    accuracy_ask_side_down = score[exchange_name]['ASK_SIDE']['TN'] / (score[exchange_name]['ASK_SIDE']['TN'] + score[exchange_name]['ASK_SIDE']['FN'])

    print(f"{exchange_name} has accuracy of {accuracy_bid_side_up} for updwards correction on the bid side")
    print(f"{exchange_name} has accuracy of {accuracy_ask_side_up} for downwards correction on the ask side")

    print(f"{exchange_name} has accuracy of {accuracy_bid_side_down} for updwards correction on the bid side")
    print(f"{exchange_name} has accuracy of {accuracy_ask_side_down} for downwards correction on the ask side")
    print("\n")

## Using a Multionmial Logit Model (Section 6)

In [ ]:
# The exchange names as hinted in footnote 1 of our paper
DF_NAMES = ['binance', 'bitmart', 'bybit', 'hashkey', 'htx', 'bitget', 'deribit', 'kuma', 'hyperliquid']
DF_NAMES_CLEAN = ['binance', 'bitmart', 'bybit', 'htx', 'bitget', 'deribit', 'kuma', 'hyperliquid']

# Keep track of the performance of every exchange respectively
score = {}
for exchange_name in DF_NAMES_CLEAN:
    score[exchange_name] = {}
    score[exchange_name]['BID_SIDE'] = {}
    score[exchange_name]['BID_SIDE']['TP'] = 0
    score[exchange_name]['BID_SIDE']['FP'] = 0
    score[exchange_name]['BID_SIDE']['TN'] = 0
    score[exchange_name]['BID_SIDE']['FN'] = 0

    score[exchange_name]['ASK_SIDE'] = {}
    score[exchange_name]['ASK_SIDE']['TP'] = 0
    score[exchange_name]['ASK_SIDE']['FP'] = 0
    score[exchange_name]['ASK_SIDE']['TN'] = 0
    score[exchange_name]['ASK_SIDE']['FN'] = 0

In [ ]:
# Window size
H = 50_000
# Train percentage
RHO = 0.7
# Max total length of the data we look at
max_j = 1_000_000 # pre determined number of rows of the file
# The stepsize
STEPSIZE = int((1-RHO)*H)
# Set the lag once
LAG = 20
# Set the rank once
RANK = 7 # K - 1
# Set threshold delta
DELTA = 0.0005
# Path
PATH = "/Downloads/data.csv"
# Adaptive group lasso solvers
group_lasso_prox_rank_b = GroupLassoProxRankSelection()
group_lasso_prox_rank_a = GroupLassoProxRankSelection()
group_lasso_prox_b = GroupLassoProxLagSelection()
group_lasso_prox_a = GroupLassoProxLagSelection()
# Define tau rank, tau lag and tau ridge and gamma
tau_rank_b = 1e10
tau_rank_a = 1e10
tau_lag_b = 0.01
tau_lag_a = 0.01
gamma = 2

# Define lookback of clusters
D = 20
kappa = 0.7

for j in range(0, max_j, STEPSIZE):
    try:
        # This should be the path to the data_1.csv file on your local machine
        skip_rows = j
        n_rows = H

        VECM_model_b = VECMModelHD()
        VECM_model_a = VECMModelHD()
        helper_methods = Helpers()

        binance, bitmart, bybit, hashkey, htx, bitget, deribit, kuma, hyperliquid = helper_methods.read_csv_to_dataframe(PATH, skip_rows, n_rows, DF_NAMES)
        # We will skip hashkey exchange
        Y_b = VECM_model_b.construct_Y([binance.bids, bitmart.bids, bybit.bids, htx.bids, bitget.bids, deribit.bids, kuma.bids, hyperliquid.bids])
        Y_a = VECM_model_a.construct_Y([binance.asks, bitmart.asks, bybit.asks, htx.asks, bitget.asks, deribit.asks, kuma.asks, hyperliquid.asks])
        # Train test split
        Y_train_b, Y_test_b = VECM_model_b.train_test_split_Y(Y_b, RHO)
        Y_train_a, Y_test_a = VECM_model_a.train_test_split_Y(Y_a, RHO)
        # Set the lag
        VECM_model_b.lag = LAG
        VECM_model_a.lag = LAG
        # Build the VECM matrices
        VECM_model_b.build_vecm_matrices(Y_train_b)
        VECM_model_a.build_vecm_matrices(Y_train_a)
        # Build frisch waugh rank matrices
        dY_tilde_b, Y1_tilde_b, Pi_tilde_b = VECM_model_b.frisch_waugh_rank_matrices()
        dY_tilde_a, Y1_tilde_a, Pi_tilde_a = VECM_model_a.frisch_waugh_rank_matrices()

        # Compute qr decomp
        Q_hat_b, R_tilde_b, Perm_b = VECM_model_b.qr_decomp(Pi_tilde_b.T)
        Q_hat_a, R_tilde_a, Perm_a = VECM_model_a.qr_decomp(Pi_tilde_a.T)
        # Compute permuation matrices
        Pmat_b = np.eye(Perm_b.shape[0])[:, Perm_b]
        Pmat_a = np.eye(Perm_a.shape[0])[:, Perm_a]
        # Compute mu_tildes
        mu_tilde_b = VECM_model_b.compute_mu_tilde(R_tilde_b)
        mu_tilde_a = VECM_model_a.compute_mu_tilde(R_tilde_a)
        # Rank pre estimate
        r_hat_qr_decomp_b = VECM_model_b.rank_pre_estimate(mu_tilde_b)
        r_hat_qr_decomp_a = VECM_model_a.rank_pre_estimate(mu_tilde_a)
        # Betas
        beta_b = VECM_model_b.get_beta(Q_hat_b, r_hat_qr_decomp_b)
        beta_a = VECM_model_a.get_beta(Q_hat_a, r_hat_qr_decomp_a)
        # Regressor matrices
        dY_tilde_b = Pmat_b.T @ dY_tilde_b
        dY_tilde_a = Pmat_a.T @ dY_tilde_a

        Y1_tilde_b = Pmat_b.T @ Y1_tilde_b
        Y1_tilde_a = Pmat_a.T @ Y1_tilde_a

        T = VECM_model_b.T_eff
        K = VECM_model_b.K

        weights = group_lasso_prox_rank_b.compute_weights(mu_tilde_b, gamma)
        weights = group_lasso_prox_rank_b.scale_weights(weights)
        Y, Z = group_lasso_prox_rank_b.construct_Y_Z_matrices(Q_hat_b, Y1_tilde_b, dY_tilde_b)

        R_hat_b, loss_history = group_lasso_prox_rank_b.fit(R_tilde_b, Y, Z, tau_rank_b, weights, tol = 1e-20)

        Sigma_w_b = VECM_model_b.residual_covariance(dY_tilde_b, R_hat_b.T @ Q_hat_b.T @ Y1_tilde_b)

        group_lasso_prox_rank_a = GroupLassoProxRankSelection()
        weights = group_lasso_prox_rank_a.compute_weights(mu_tilde_a, gamma)
        weights = group_lasso_prox_rank_a.scale_weights(weights)
        Y, Z = group_lasso_prox_rank_a.construct_Y_Z_matrices(Q_hat_a, Y1_tilde_a, dY_tilde_a)
        R_hat_a, loss_history = group_lasso_prox_rank_a.fit(R_tilde_a, Y, Z, tau_rank_a, weights, tol = 1e-20)

        Sigma_w_a = VECM_model_a.residual_covariance(dY_tilde_a, R_hat_a.T @ Q_hat_a.T @ Y1_tilde_a)

        # Set rank
        VECM_model_b.rank = RANK
        VECM_model_a.rank = RANK
        # Get alphas
        alpha_b = VECM_model_b.get_alpha(Pmat_b, R_hat_b, r_hat_qr_decomp_b)
        alpha_a = VECM_model_a.get_alpha(Pmat_a, R_hat_a, r_hat_qr_decomp_a)
        # Frisch waugh lag matrices
        dY_check_b, dX_check_b = VECM_model_b.frisch_waugh_lag_matrices()
        dY_check_a, dX_check_a = VECM_model_a.frisch_waugh_lag_matrices()
        # Gammas pre estimates
        Gamma_check_b = VECM_model_b.least_squares_estimate_Gamma(dX_check_b, dY_check_b)
        Gamma_check_a = VECM_model_a.least_squares_estimate_Gamma(dX_check_a, dY_check_a)
        c_b = 1
        c_a = 1
        tau_ridge_b = c_b * np.log(T)
        tau_ridge_a = c_a * np.log(T)
        Gamma_tilde_b = VECM_model_b.ridge_estimator_Gamma(dX_check_b, dY_check_b, tau_ridge_b, LAG)
        Gamma_tilde_a = VECM_model_a.ridge_estimator_Gamma(dX_check_a, dY_check_a, tau_ridge_a, LAG)

        # lag estimation
        K = VECM_model_b.K
        weights = group_lasso_prox_b.compute_weights(Gamma_tilde_b, LAG, K, gamma)
        weights = group_lasso_prox_b.scale_weights(weights)
        Y, Z = group_lasso_prox_b.helper_loss(LAG, dY_check_b)
        Gamma_hat_b, loss_history = group_lasso_prox_b.fit(Gamma_tilde_b, Y, Z, LAG, K, tau_lag_b, weights)
        Sigma_w_b = VECM_model_b.residual_covariance(dY_check_b, Gamma_hat_b @ dX_check_b)


        weights = group_lasso_prox_a.compute_weights(Gamma_tilde_a, LAG, K, gamma)
        weights = group_lasso_prox_a.scale_weights(weights)
        Y, Z = group_lasso_prox_a.helper_loss(LAG, dY_check_a)
        Gamma_hat_a, loss_history = group_lasso_prox_a.fit(Gamma_tilde_a, Y, Z, LAG, K, tau_lag_a, weights)
        Sigma_w_a = VECM_model_a.residual_covariance(dY_check_a, Gamma_hat_a @ dX_check_a)

        # set gamma and lag
        Gammas_b = Gamma_hat_b[:, :VECM_model_b.lag*K]
        Gammas_a = Gamma_hat_a[:, :VECM_model_a.lag*K]

        VECM_model_b.lag = LAG
        VECM_model_a.lag = LAG

        # Compute the orthogonal compoents of alpha and beta
        alpha_perp_b = VECM_model_b.compute_null_space_basis(alpha_b.T)
        beta_perp_b = VECM_model_b.compute_null_space_basis(beta_b.T)
        alpha_perp_a = VECM_model_a.compute_null_space_basis(alpha_a.T)
        beta_perp_a = VECM_model_a.compute_null_space_basis(beta_a.T)
        # First evaluate the latent efficient price from Section 4.2 of our paper
        persistent_component_b_train, transitory_component_b_train = VECM_model_b.compute_P_T_decomposition(alpha_b, beta_b, alpha_perp_b, beta_perp_b, Y_train_b)
        persistent_component_a_train, transitory_component_a_train = VECM_model_a.compute_P_T_decomposition(alpha_b, beta_b, alpha_perp_a, beta_perp_a, Y_train_a)

        persistent_component_b_test, transitory_component_b_test = VECM_model_b.compute_P_T_decomposition(alpha_b, beta_b, alpha_perp_b, beta_perp_b, Y_test_b)
        persistent_component_a_test, transitory_component_a_test = VECM_model_a.compute_P_T_decomposition(alpha_b, beta_b, alpha_perp_a, beta_perp_a, Y_test_a)


        # Loop over all variables to compute the score respectively
        for idx, exchange_name in zip(range(len(DF_NAMES_CLEAN)), DF_NAMES_CLEAN):
            # Only test bid series here
            multionmial_logit_b = MultinomialLogit()
            multionmial_logit_a = MultinomialLogit()

            # Define leader and lagger series
            # x is leading y
            # Define train data
            x_series_train_b = pd.Series((persistent_component_b_train)[idx])
            y_series_train_b = pd.Series(Y_train_b[idx])

            x_series_train_a = pd.Series((persistent_component_a_train)[idx])
            y_series_train_a = pd.Series(Y_train_a[idx])

            # Define test data
            x_series_test_b = pd.Series(persistent_component_b_test[idx])
            y_series_test_b = pd.Series(Y_test_b[idx])

            x_series_test_a = pd.Series(persistent_component_a_test[idx])
            y_series_test_a = pd.Series(Y_test_a[idx])

            # Get the data frame with cluster ids
            cluster_df_train_b = multionmial_logit_b.compute_static_cluster_returns(x_series_train_b, y_series_train_b)
            cluster_df_test_b = multionmial_logit_b.compute_static_cluster_returns(x_series_test_b, y_series_test_b)

            cluster_df_train_a = multionmial_logit_a.compute_static_cluster_returns(x_series_train_a, y_series_train_a)
            cluster_df_test_a = multionmial_logit_a.compute_static_cluster_returns(x_series_test_a, y_series_test_a)

            # Generate the training data and fit the model with the train data we already have
            # Lookback length for clusters
            X_train_b, y_train_b = multionmial_logit_b.generate_model_data(cluster_df_train_b, D)
            X_test_b, y_test_b = multionmial_logit_b.generate_model_data(cluster_df_test_b, D)

            X_train_a, y_train_a = multionmial_logit_a.generate_model_data(cluster_df_train_a, D)
            X_test_a, y_test_a = multionmial_logit_a.generate_model_data(cluster_df_test_a, D)

            # Fit models
            model_b = multionmial_logit_b.fit(X_train_b, y_train_b)
            model_a = multionmial_logit_a.fit(X_train_a, y_train_a)
            
            # Check the performance

            for i in range(len(X_test_b)):
                prediction = model_b.predict(X_test_b[i])[0]
                true_value = np.sign(y_test_b[i])

                # down prediction
            
                # up prediction
                if np.max(prediction) == prediction[2] and np.max(prediction) >= kappa:
                    if true_value == +1:
                        score[exchange_name]['BID_SIDE']['TP'] += 1
                    else:
                        score[exchange_name]['BID_SIDE']['FP'] += 1

                elif np.max(prediction) == prediction[0] and np.max(prediction) >= kappa:
                    if true_value == -1:
                        score[exchange_name]['BID_SIDE']['TN'] += 1
                    else:
                        score[exchange_name]['BID_SIDE']['FN'] += 1

                else:
                    pass

            for i in range(len(X_test_a)):
                prediction = model_a.predict(X_test_a[i])[0]
                true_value = np.sign(y_test_a[i])

                # down prediction
            
                # up prediction
                if np.max(prediction) == prediction[2] and np.max(prediction) >= kappa:
                    if true_value == +1:
                        score[exchange_name]['ASK_SIDE']['TP'] += 1
                    else:
                        score[exchange_name]['ASK_SIDE']['FP'] += 1

                elif np.max(prediction) == prediction[0] and np.max(prediction) >= kappa:
                    if true_value == -1:
                        score[exchange_name]['BID_SIDE']['TN'] += 1
                    else:
                        score[exchange_name]['BID_SIDE']['FN'] += 1

                else:
                    pass

        print(f"###{j / max_j}###")

    except Exception as exp:
        # ignore warnings, sometimes labels are only +1 and -1 due to the data being poorly scaled
        # need to optimize a bit here
        print(f"Something went wrong: {exp}")
        continue

In [ ]:
score['kuma']